A Data-Driven Market Intelligence System for Stock Risk and Behavior Analysis at the Nairobi Securities Exchange

Problem Statement

Despite the Nairobi Securities Exchange (NSE) seeing a surge in equity turnover-rising 18% to Ksh 56 billion in 2025 ([SE Half-Year Results, 2025](https://www.nse.co.ke/wp-content/uploads/NSE-Plc-Unaudited-Group-results-for-the-6-months-ended-30-June-2025.pdf)), retail participation remains hampered by a critical information gap. Research from the Institute of Certified Investment and Financial Analysts reveals that 77% of Kenyan retail investors rely on "personal research" and social intuition because they lack accessible analytical tools [ICIFA Annual Report, 2024](https://icifa.co.ke/static/resources/others/annual-report-2024465e3dbed42d.pdf)).

While the market added over Ksh 1 Trillion in capitalization since 2023, most investors suffer from "herding behavior," where decisions are made by following the crowd rather than technical data [USIU-Africa Research, 2025](https://erepo.usiu.ac.ke/xmlui/bitstream/handle/11732/8460/MASILA%20BRIAN%20SALU%20MBA%202024.pdf?sequence=1&isAllowed=y). This project bridges this gap by converting raw daily prices into behavioral risk clusters, moving investors from intuition to evidence-based decision-making.

## OBJECTIVES

**Main Objective**
- To develop a data-driven stock market intelligence system for the Nairobi Securities Exchange that analyzes historical stock price behavior and sector characteristics to support informed and risk-aware investment decisions.

**Specific Objectives**
1) Feature Engineering: To derive financial metrics including Rolling Volatility, Daily Returns, and Maximum Drawdowns to quantify stock behavior.

2) Behavioral Segmentation: To apply Unsupervised Machine Learning (K-Means/DBSCAN) to group stocks into risk-based clusters (e.g., Stable, High-Volatility, or Speculative).

3) Sector Risk Analysis: To identify systemic risks and stability patterns across different market sectors.

4) Interactive Deployment: To present these insights through a Streamlit Dashboard that allows users to select stocks, view their risk profile, and compare them against their sectors.

In [89]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans, DBSCAN
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score
import warnings
warnings.filterwarnings('ignore')

In [55]:
df_2021 = pd.read_csv("../Data/NSE_data_all_stocks_2021_upto_31dec2021.csv")
df_2022 = pd.read_csv("../Data/NSE_data_all_stocks_2022.csv")
df_2023 = pd.read_csv("../Data/NSE_data_all_stocks_2023.csv")
df_2024 = pd.read_csv("../Data/NSE_data_all_stocks_2024.csv")


from IPython.display import display

display(df_2021.head())
print(" " * 2)
display(df_2022.head())
print(" " * 2)
display(df_2023.head())
print(" " * 2)
display(df_2024.head())

,DATE,CODE,NAME,12m Low,12m High,Day Low,Day High,Day Price,Previous,Change,Change%,Volume,Adjust
0,04-Jan-21,EGAD,Eaagads Ltd,8.2,14,12.5,12.5,12.5,12.5,-,-,"3,200",-
1,04-Jan-21,KUKZ,Kakuzi Plc,300,397,365,365,365,365,-,-,-,-
2,04-Jan-21,KAPC,Kapchorua Tea Kenya Plc,59,90,78,78,78,78,-,-,-,-
3,04-Jan-21,LIMT,Limuru Tea Plc,360,475,360,360,360,360,-,-,100,-
4,04-Jan-21,SASN,Sasini Plc,14.8,20,19.5,19.5,19.5,19.5,-,-,-,-


,Date,Code,Name,12m Low,12m High,Day Low,Day High,Day Price,Previous,Change,Change%,Volume,Adjusted Price
0,3-Jan-22,EGAD,Eaagads Ltd,10,15,13.5,13.8,13.5,13.5,-,-,"4,000",-
1,3-Jan-22,KUKZ,Kakuzi Plc,355,427,385,385,385,385,-,-,-,-
2,3-Jan-22,KAPC,Kapchorua Tea Kenya Plc,80,101,99.5,99.5,99.5,95.5,4,4.19%,100,-
3,3-Jan-22,LIMT,Limuru Tea Plc,260,360,320,320,320,320,-,-,-,-
4,3-Jan-22,SASN,Sasini Plc,16.75,22.6,18.7,18.7,18.7,18.7,-,-,-,-


,Date,Code,Name,12m Low,12m High,Day Low,Day High,Day Price,Previous,Change,Change%,Volume,Adjusted Price
0,3-Jan-23,EGAD,Eaagads Ltd,10.35,14.5,10.5,10.5,10.5,10.5,-,-,"1,900.00",-
1,3-Jan-23,KUKZ,Kakuzi Plc,342,440,385,385,385,385,-,-,-,-
2,3-Jan-23,KAPC,Kapchorua Tea Kenya Plc,207,280,115.75,115.75,115.75,113.25,2.5,2.21%,100,-
3,3-Jan-23,LIMT,Limuru Tea Plc,365,380,420,420,420,420,-,-,-,-
4,3-Jan-23,SASN,Sasini Plc,15.1,22,22,22.5,22.45,22.45,-,-,"6,900.00",-


,Date,Code,Name,12m Low,12m High,Day Low,Day High,Day Price,Previous,Change,Change%,Volume,Adjusted Price
0,2-Jan-24,EGAD,Eaagads Ltd,10.35,14.5,12.8,12.8,12.8,13.95,-1.15,-8.24%,100,-
1,2-Jan-24,KUKZ,Kakuzi Plc,342,440,385,385,385,385,-,-,-,-
2,2-Jan-24,KAPC,Kapchorua Tea Kenya Plc,207,280,215,215,215,215,-,-,-,-
3,2-Jan-24,LIMT,Limuru Tea Plc,365,380,380,380,380,380,-,-,-,-
4,2-Jan-24,SASN,Sasini Plc,15.1,22,20,20,20,20,-,-,"3,300.00",-


**Date** – The calendar day of the trading information.

**Stock Code** – A unique identifier (ticker) for the stock.

**Stock Name** – The company’s name.

**12-month Low price** – The lowest price the stock traded at over the last 12 months.

**12-month High price** – The highest price over the last 12 months.

**Day’s Low price**– Lowest price the stock traded at on that day.

**Day’s High price** – Highest price on that day.

**Day’s Final Price** – Closing price that day.

**Previous traded price** – Closing price from the previous trading day.

**Change in price value** – Difference between today’s and previous day’s price.

**Change in price %** – Percentage change.

**Volume traded** – Number of shares traded that day.

**Adjusted price** – Price adjusted for corporate actions (like dividends, splits).

**Sector** -the sector the company belongs to

Mapping 2021 column names to match 2022-2024

In [56]:
# Renaming 2021 columns to match 2022-2024
df_2021 = df_2021.rename(columns={
    'DATE': 'Date',
    'CODE': 'Code',
    'NAME': 'Name',
    'Adjust': 'Adjusted Price'
})

df_2021.head()

,Date,Code,Name,12m Low,12m High,Day Low,Day High,Day Price,Previous,Change,Change%,Volume,Adjusted Price
0,04-Jan-21,EGAD,Eaagads Ltd,8.2,14,12.5,12.5,12.5,12.5,-,-,"3,200",-
1,04-Jan-21,KUKZ,Kakuzi Plc,300,397,365,365,365,365,-,-,-,-
2,04-Jan-21,KAPC,Kapchorua Tea Kenya Plc,59,90,78,78,78,78,-,-,-,-
3,04-Jan-21,LIMT,Limuru Tea Plc,360,475,360,360,360,360,-,-,100,-
4,04-Jan-21,SASN,Sasini Plc,14.8,20,19.5,19.5,19.5,19.5,-,-,-,-


In [57]:
df_2021.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17746 entries, 0 to 17745
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Date            17746 non-null  object
 1   Code            17746 non-null  object
 2   Name            17746 non-null  object
 3   12m Low         17746 non-null  object
 4   12m High        17746 non-null  object
 5   Day Low         17746 non-null  object
 6   Day High        17746 non-null  object
 7   Day Price       17746 non-null  object
 8   Previous        17746 non-null  object
 9   Change          17746 non-null  object
 10  Change%         17746 non-null  object
 11  Volume          17746 non-null  object
 12  Adjusted Price  17746 non-null  object
dtypes: object(13)
memory usage: 1.8+ MB


In [58]:
df_2021.isna().sum()

Date              0
Code              0
Name              0
12m Low           0
12m High          0
Day Low           0
Day High          0
Day Price         0
Previous          0
Change            0
Change%           0
Volume            0
Adjusted Price    0
dtype: int64

In [59]:
df_2022.isna().sum() 

Date              0
Code              0
Name              0
12m Low           0
12m High          0
Day Low           0
Day High          0
Day Price         0
Previous          0
Change            0
Change%           0
Volume            0
Adjusted Price    0
dtype: int64

In [60]:
df_2023.isna().sum()  

Date              0
Code              0
Name              0
12m Low           0
12m High          0
Day Low           0
Day High          0
Day Price         0
Previous          0
Change            0
Change%           0
Volume            0
Adjusted Price    0
dtype: int64

In [61]:
df_2024.isna().sum()   

Date              0
Code              0
Name              0
12m Low           0
12m High          0
Day Low           0
Day High          0
Day Price         0
Previous          0
Change            0
Change%           0
Volume            0
Adjusted Price    0
dtype: int64

In [62]:
print(len(df_2021))
print(len(df_2022))
print(len(df_2023))
print(len(df_2024))

17746
16806
17274
18119


Sectors

In [63]:
sectors_2021 = pd.read_csv("../Data/NSE_data_stock_market_sectors_as_at_31dec2021.csv")
sectors_2022 = pd.read_csv("../Data/NSE_data_stock_market_sectors_2022.csv")
sectors_2023_2024 = pd.read_csv("../Data/NSE_data_stock_market_sectors_2023_2024.csv")


print("Sector data shapes:")
print(f"2021: {sectors_2021.shape}")
print(f"2022: {sectors_2022.shape}")
print(f"2023-2024: {sectors_2023_2024.shape}")

Sector data shapes:
2021: (71, 3)
2022: (71, 3)
2023-2024: (73, 3)


In [64]:
sectors_2021.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71 entries, 0 to 70
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   SECTOR  71 non-null     object
 1   CODE    71 non-null     object
 2   NAME    71 non-null     object
dtypes: object(3)
memory usage: 1.8+ KB


In [65]:
sectors_2022.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71 entries, 0 to 70
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Sector      71 non-null     object
 1   Stock_code  71 non-null     object
 2   Stock_name  71 non-null     object
dtypes: object(3)
memory usage: 1.8+ KB


In [66]:
sectors_2023_2024.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 73 entries, 0 to 72
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Sector      73 non-null     object
 1   Stock_code  73 non-null     object
 2   Stock_name  72 non-null     object
dtypes: object(3)
memory usage: 1.8+ KB


Merging Stock Data with Sector Information

In [67]:
# Standardizing ALL columns in sector files for consistency
sectors_2021 = sectors_2021.rename(columns={
    'SECTOR': 'Sector', 
    'CODE': 'Code', 
    'NAME': 'Name'
})

sectors_2022 = sectors_2022.rename(columns={
    'Sector': 'Sector',
    'Stock_code': 'Code', 
    'Stock_name': 'Name'
})

sectors_2023_2024 = sectors_2023_2024.rename(columns={
    'Sector': 'Sector',
    'Stock_code': 'Code', 
    'Stock_name': 'Name'
})

# Merging 
df_2021_final = df_2021.merge(sectors_2021[['Code', 'Sector']], on='Code', how='left')
df_2022_final = df_2022.merge(sectors_2022[['Code', 'Sector']], on='Code', how='left')
df_2023_final = df_2023.merge(sectors_2023_2024[['Code', 'Sector']], on='Code', how='left')
df_2024_final = df_2024.merge(sectors_2023_2024[['Code', 'Sector']], on='Code', how='left')

In [68]:
# concatenating all years together
all_stocks = pd.concat([df_2021_final, df_2022_final, df_2023_final, df_2024_final], ignore_index=True)

all_stocks.tail()

,Date,Code,Name,12m Low,12m High,Day Low,Day High,Day Price,Previous,Change,Change%,Volume,Adjusted Price,Sector
69940,31-Dec-24,^N25I,NSE 25-Share Index,2364.29,"3,457.87","3,402.80","3,402.80","3,402.80","3,380.24",22.56,0.67%,-,-,Indices
69941,31-Dec-24,^NASI,NSE All-Share Index,90.1,125.34,123.48,123.48,123.48,123.12,0.36,0.29%,-,-,Indices
69942,31-Dec-24,^NBDI,NSE Bonds Index,986.46,"1,108.71","1,107.41","1,107.41","1,107.41","1,093.87",13.54,1.24%,-,-,NaN
69943,31-Dec-24,^ZKEQTK,Zamara Kenya Equity Index (KES),1535.23,"2,547.58","2,519.80","2,519.80","2,519.80","2,495.28",24.52,0.98%,-,-,Indices
69944,31-Dec-24,^ZKEQTU,Zamara Kenya Equity Index (USD),818.46,"1,697.07","1,677.27","1,677.27","1,677.27","1,662.24",15.03,0.90%,-,-,Indices


In [69]:
all_stocks.shape

(69945, 14)

DATA CLEANING

In [70]:
all_stocks.head(10)

,Date,Code,Name,12m Low,12m High,Day Low,Day High,Day Price,Previous,Change,Change%,Volume,Adjusted Price,Sector
0,04-Jan-21,EGAD,Eaagads Ltd,8.2,14,12.5,12.5,12.5,12.5,-,-,"3,200",-,Agricultural
1,04-Jan-21,KUKZ,Kakuzi Plc,300,397,365,365,365,365,-,-,-,-,Agricultural
2,04-Jan-21,KAPC,Kapchorua Tea Kenya Plc,59,90,78,78,78,78,-,-,-,-,Agricultural
3,04-Jan-21,LIMT,Limuru Tea Plc,360,475,360,360,360,360,-,-,100,-,Agricultural
4,04-Jan-21,SASN,Sasini Plc,14.8,20,19.5,19.5,19.5,19.5,-,-,-,-,Agricultural
5,04-Jan-21,WTK,Williamson Tea Kenya Plc,92,150,127.5,130,129.75,130.25,-0.5,0.38%,"3,200",-,Agricultural
6,04-Jan-21,CGEN,Car and General (K) Ltd,20,26,22,22,22,22,-,-,-,-,Automobiles and Accessories
7,04-Jan-21,ABSA,ABSA Bank Kenya Plc,8.5,14.2,9.42,9.8,9.52,9.66,-0.14,1.45%,"18,500",-,Banking
8,04-Jan-21,BKG,BK Group Plc,12.15,28,20.5,20.5,20.5,20.5,-,-,-,-,Banking
9,04-Jan-21,COOP,Co-operative Bank of Kenya Ltd,10.2,17,12.2,13.7,12.55,12.5,0.05,0.40%,"248,100",-,Banking


In [71]:
all_stocks.tail(10)

,Date,Code,Name,12m Low,12m High,Day Low,Day High,Day Price,Previous,Change,Change%,Volume,Adjusted Price,Sector
69935,31-Dec-24,SCOM,Safaricom Plc,12.95,21.15,17,17.35,17.05,17.15,-0.1,-0.58%,"1,243,600.00",-,Telecommunication
69936,31-Dec-24,LAPR,Laptrust Imara Income-REIT,20,20,20,20,20,20,-,-,-,-,Real Estate Investment Trusts
69937,31-Dec-24,GLD,ABSA NewGold ETF,2625,"3,330.00","3,260.00","3,260.00","3,260.00","3,260.00",-,-,-,-,Exchange Traded Funds
69938,31-Dec-24,^N10I,NSE 10-Share Index,897.19,"1,327.40","1,302.31","1,302.31","1,302.31","1,290.22",12.09,0.94%,-,-,Indices
69939,31-Dec-24,^N20I,NSE 20-Share Index,1487.89,"18,845.31","2,010.65","2,010.65","2,010.65","1,993.74",16.91,0.85%,-,-,Indices
69940,31-Dec-24,^N25I,NSE 25-Share Index,2364.29,"3,457.87","3,402.80","3,402.80","3,402.80","3,380.24",22.56,0.67%,-,-,Indices
69941,31-Dec-24,^NASI,NSE All-Share Index,90.1,125.34,123.48,123.48,123.48,123.12,0.36,0.29%,-,-,Indices
69942,31-Dec-24,^NBDI,NSE Bonds Index,986.46,"1,108.71","1,107.41","1,107.41","1,107.41","1,093.87",13.54,1.24%,-,-,NaN
69943,31-Dec-24,^ZKEQTK,Zamara Kenya Equity Index (KES),1535.23,"2,547.58","2,519.80","2,519.80","2,519.80","2,495.28",24.52,0.98%,-,-,Indices
69944,31-Dec-24,^ZKEQTU,Zamara Kenya Equity Index (USD),818.46,"1,697.07","1,677.27","1,677.27","1,677.27","1,662.24",15.03,0.90%,-,-,Indices


In [72]:
all_stocks.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 69945 entries, 0 to 69944
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Date            69945 non-null  object
 1   Code            69945 non-null  object
 2   Name            69945 non-null  object
 3   12m Low         69945 non-null  object
 4   12m High        69945 non-null  object
 5   Day Low         69945 non-null  object
 6   Day High        69945 non-null  object
 7   Day Price       69945 non-null  object
 8   Previous        69945 non-null  object
 9   Change          69945 non-null  object
 10  Change%         69945 non-null  object
 11  Volume          69945 non-null  object
 12  Adjusted Price  69945 non-null  object
 13  Sector          69754 non-null  object
dtypes: object(14)
memory usage: 7.5+ MB


In [73]:
all_stocks.isna()

,Date,Code,Name,12m Low,12m High,Day Low,Day High,Day Price,Previous,Change,Change%,Volume,Adjusted Price,Sector
0,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69940,False,False,False,False,False,False,False,False,False,False,False,False,False,False
69941,False,False,False,False,False,False,False,False,False,False,False,False,False,False
69942,False,False,False,False,False,False,False,False,False,False,False,False,False,True
69943,False,False,False,False,False,False,False,False,False,False,False,False,False,False


In [74]:
all_stocks['Sector'].isna().sum()


191

In [75]:
#check percentage
all_stocks['Sector'].isna().mean() * 100


0.27307169919222246

In [76]:
all_stocks = all_stocks.dropna(subset=['Sector'])


In [77]:
all_stocks.info()

<class 'pandas.core.frame.DataFrame'>
Index: 69754 entries, 0 to 69944
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Date            69754 non-null  object
 1   Code            69754 non-null  object
 2   Name            69754 non-null  object
 3   12m Low         69754 non-null  object
 4   12m High        69754 non-null  object
 5   Day Low         69754 non-null  object
 6   Day High        69754 non-null  object
 7   Day Price       69754 non-null  object
 8   Previous        69754 non-null  object
 9   Change          69754 non-null  object
 10  Change%         69754 non-null  object
 11  Volume          69754 non-null  object
 12  Adjusted Price  69754 non-null  object
 13  Sector          69754 non-null  object
dtypes: object(14)
memory usage: 8.0+ MB


In [78]:
all_stocks.duplicated().sum()

0

In [79]:
all_stocks['Date'] = pd.to_datetime(all_stocks['Date'], errors='coerce')

<ipython-input-79-24181395f29f>:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  all_stocks['Date'] = pd.to_datetime(all_stocks['Date'], errors='coerce')


In [80]:
all_stocks.info()

<class 'pandas.core.frame.DataFrame'>
Index: 69754 entries, 0 to 69944
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Date            69754 non-null  datetime64[ns]
 1   Code            69754 non-null  object        
 2   Name            69754 non-null  object        
 3   12m Low         69754 non-null  object        
 4   12m High        69754 non-null  object        
 5   Day Low         69754 non-null  object        
 6   Day High        69754 non-null  object        
 7   Day Price       69754 non-null  object        
 8   Previous        69754 non-null  object        
 9   Change          69754 non-null  object        
 10  Change%         69754 non-null  object        
 11  Volume          69754 non-null  object        
 12  Adjusted Price  69754 non-null  object        
 13  Sector          69754 non-null  object        
dtypes: datetime64[ns](1), object(13)
memory usage: 8.0+ MB


In [81]:
# % change where '-' is present
all_stocks[all_stocks['Change%'] == '-']

,Date,Code,Name,12m Low,12m High,Day Low,Day High,Day Price,Previous,Change,Change%,Volume,Adjusted Price,Sector
0,2021-01-04,EGAD,Eaagads Ltd,8.2,14,12.5,12.5,12.5,12.5,-,-,"3,200",-,Agricultural
1,2021-01-04,KUKZ,Kakuzi Plc,300,397,365,365,365,365,-,-,-,-,Agricultural
2,2021-01-04,KAPC,Kapchorua Tea Kenya Plc,59,90,78,78,78,78,-,-,-,-,Agricultural
3,2021-01-04,LIMT,Limuru Tea Plc,360,475,360,360,360,360,-,-,100,-,Agricultural
4,2021-01-04,SASN,Sasini Plc,14.8,20,19.5,19.5,19.5,19.5,-,-,-,-,Agricultural
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69927,2024-12-31,BOC,BOC Kenya Plc,67,96,88.75,88.75,88.75,88.75,-,-,-,-,Manufacturing and Allied
69932,2024-12-31,ORCH,Kenya Orchards Ltd,19,77,70,70,70,70,-,-,-,-,Manufacturing and Allied
69933,2024-12-31,MSC,Mumias Sugar Company Ltd,0.27,0.27,0.27,0.27,0.27,0.27,-,-,-,-,Manufacturing and Allied
69936,2024-12-31,LAPR,Laptrust Imara Income-REIT,20,20,20,20,20,20,-,-,-,-,Real Estate Investment Trusts


In [82]:
cols = [
    "12m Low","12m High","Day Low","Day High",
    "Day Price","Previous","Change","Change%", "Volume","Adjusted Price"
]

# remove % first
all_stocks["Change%"] = all_stocks["Change%"].str.replace("%", "", regex=False)

# remove thousand separators + replace '-' + cast
all_stocks[cols] = (
    all_stocks[cols]
        .replace(",", "", regex=True)
        .replace("-", 0)
        .apply(pd.to_numeric, errors="coerce"))

In [83]:
all_stocks.head(10)

,Date,Code,Name,12m Low,12m High,Day Low,Day High,Day Price,Previous,Change,Change%,Volume,Adjusted Price,Sector
0,2021-01-04,EGAD,Eaagads Ltd,8.20,14.0,12.50,12.5,12.50,12.50,0.00,0.00,3200.0,0.0,Agricultural
1,2021-01-04,KUKZ,Kakuzi Plc,300.00,397.0,365.00,365.0,365.00,365.00,0.00,0.00,0.0,0.0,Agricultural
2,2021-01-04,KAPC,Kapchorua Tea Kenya Plc,59.00,90.0,78.00,78.0,78.00,78.00,0.00,0.00,0.0,0.0,Agricultural
3,2021-01-04,LIMT,Limuru Tea Plc,360.00,475.0,360.00,360.0,360.00,360.00,0.00,0.00,100.0,0.0,Agricultural
4,2021-01-04,SASN,Sasini Plc,14.80,20.0,19.50,19.5,19.50,19.50,0.00,0.00,0.0,0.0,Agricultural
5,2021-01-04,WTK,Williamson Tea Kenya Plc,92.00,150.0,127.50,130.0,129.75,130.25,-0.50,0.38,3200.0,0.0,Agricultural
6,2021-01-04,CGEN,Car and General (K) Ltd,20.00,26.0,22.00,22.0,22.00,22.00,0.00,0.00,0.0,0.0,Automobiles and Accessories
7,2021-01-04,ABSA,ABSA Bank Kenya Plc,8.50,14.2,9.42,9.8,9.52,9.66,-0.14,1.45,18500.0,0.0,Banking
8,2021-01-04,BKG,BK Group Plc,12.15,28.0,20.50,20.5,20.50,20.50,0.00,0.00,0.0,0.0,Banking
9,2021-01-04,COOP,Co-operative Bank of Kenya Ltd,10.20,17.0,12.20,13.7,12.55,12.50,0.05,0.40,248100.0,0.0,Banking


In [84]:
all_stocks.tail(10)

,Date,Code,Name,12m Low,12m High,Day Low,Day High,Day Price,Previous,Change,Change%,Volume,Adjusted Price,Sector
69934,2024-12-31,UNGA,Unga Group Ltd,11.05,18.45,15.00,15.00,15.00,16.00,-1.00,-6.25,100.0,0.0,Manufacturing and Allied
69935,2024-12-31,SCOM,Safaricom Plc,12.95,21.15,17.00,17.35,17.05,17.15,-0.10,-0.58,1243600.0,0.0,Telecommunication
69936,2024-12-31,LAPR,Laptrust Imara Income-REIT,20.00,20.00,20.00,20.00,20.00,20.00,0.00,0.00,0.0,0.0,Real Estate Investment Trusts
69937,2024-12-31,GLD,ABSA NewGold ETF,2625.00,3330.00,3260.00,3260.00,3260.00,3260.00,0.00,0.00,0.0,0.0,Exchange Traded Funds
69938,2024-12-31,^N10I,NSE 10-Share Index,897.19,1327.40,1302.31,1302.31,1302.31,1290.22,12.09,0.94,0.0,0.0,Indices
69939,2024-12-31,^N20I,NSE 20-Share Index,1487.89,18845.31,2010.65,2010.65,2010.65,1993.74,16.91,0.85,0.0,0.0,Indices
69940,2024-12-31,^N25I,NSE 25-Share Index,2364.29,3457.87,3402.80,3402.80,3402.80,3380.24,22.56,0.67,0.0,0.0,Indices
69941,2024-12-31,^NASI,NSE All-Share Index,90.10,125.34,123.48,123.48,123.48,123.12,0.36,0.29,0.0,0.0,Indices
69943,2024-12-31,^ZKEQTK,Zamara Kenya Equity Index (KES),1535.23,2547.58,2519.80,2519.80,2519.80,2495.28,24.52,0.98,0.0,0.0,Indices
69944,2024-12-31,^ZKEQTU,Zamara Kenya Equity Index (USD),818.46,1697.07,1677.27,1677.27,1677.27,1662.24,15.03,0.90,0.0,0.0,Indices


FEATURE ENGINEERING

NEXT DAY MOVEMENT

In [85]:
# Shift the 'Day Price' column to get next day's price
all_stocks['Next Day Price'] = all_stocks['Day Price'].shift(-1)

# Create price trend column: 1 = up, 0 = down
all_stocks['Price Trend'] = (all_stocks['Next Day Price'] > all_stocks['Day Price']).astype(int)

#Include threshold for significant movement
#df['Price Trend'] = ((df['Next Day Price'] - df['Day Price']) / df['Day Price'] > 0.005).astype(int)

def trend(row):
    if row['Next Day Price'] > row['Day Price']:
        return 'Up'
    elif row['Next Day Price'] < row['Day Price']:
        return 'Down'
    else:
        return 'No Change'

all_stocks['Trend'] = all_stocks.apply(trend, axis=1)




all_stocks.tail(5)


,Date,Code,Name,12m Low,12m High,Day Low,Day High,Day Price,Previous,Change,Change%,Volume,Adjusted Price,Sector,Next Day Price,Price Trend,Trend
69939,2024-12-31,^N20I,NSE 20-Share Index,1487.89,18845.31,2010.65,2010.65,2010.65,1993.74,16.91,0.85,0.0,0.0,Indices,3402.80,1,Up
69940,2024-12-31,^N25I,NSE 25-Share Index,2364.29,3457.87,3402.80,3402.80,3402.80,3380.24,22.56,0.67,0.0,0.0,Indices,123.48,0,Down
69941,2024-12-31,^NASI,NSE All-Share Index,90.10,125.34,123.48,123.48,123.48,123.12,0.36,0.29,0.0,0.0,Indices,2519.80,1,Up
69943,2024-12-31,^ZKEQTK,Zamara Kenya Equity Index (KES),1535.23,2547.58,2519.80,2519.80,2519.80,2495.28,24.52,0.98,0.0,0.0,Indices,1677.27,0,Down
69944,2024-12-31,^ZKEQTU,Zamara Kenya Equity Index (USD),818.46,1697.07,1677.27,1677.27,1677.27,1662.24,15.03,0.90,0.0,0.0,Indices,NaN,0,No Change


VOLATILITY

In [87]:


# Daily returns
all_stocks['Daily Return'] = all_stocks['Day Price'].pct_change()

# Rolling volatility (5-day)
window = 5
all_stocks['Rolling Volatility'] = all_stocks['Daily Return'].rolling(window).std()

# Annualized volatility
#trading_days = 252
#all_stocks['Annualized Volatility'] = all_stocks['Rolling Volatility'] * (trading_days**0.5)

all_stocks.head(5)


,Date,Code,Name,12m Low,12m High,Day Low,Day High,Day Price,Previous,Change,Change%,Volume,Adjusted Price,Sector,Next Day Price,Price Trend,Trend,Daily Return,Rolling Volatility
0,2021-01-04,EGAD,Eaagads Ltd,8.2,14.0,12.5,12.5,12.5,12.5,0.0,0.0,3200.0,0.0,Agricultural,365.00,1,Up,NaN,NaN
1,2021-01-04,KUKZ,Kakuzi Plc,300.0,397.0,365.0,365.0,365.0,365.0,0.0,0.0,0.0,0.0,Agricultural,78.00,0,Down,28.200000,NaN
2,2021-01-04,KAPC,Kapchorua Tea Kenya Plc,59.0,90.0,78.0,78.0,78.0,78.0,0.0,0.0,0.0,0.0,Agricultural,360.00,1,Up,-0.786301,NaN
3,2021-01-04,LIMT,Limuru Tea Plc,360.0,475.0,360.0,360.0,360.0,360.0,0.0,0.0,100.0,0.0,Agricultural,19.50,0,Down,3.615385,NaN
4,2021-01-04,SASN,Sasini Plc,14.8,20.0,19.5,19.5,19.5,19.5,0.0,0.0,0.0,0.0,Agricultural,129.75,1,Up,-0.945833,NaN


Maximum Drawdowns to quantify stock behavior

In [88]:


# 1. Calculate the cumulative maximum of Day Price
all_stocks['Cumulative Max'] = all_stocks['Day Price'].cummax()

# 2. Calculate Drawdown
all_stocks['Drawdown'] = (all_stocks['Day Price'] - all_stocks['Cumulative Max']) / all_stocks['Cumulative Max']

# 3. Maximum Drawdown so far (rolling)
all_stocks['Max Drawdown So Far'] = all_stocks['Drawdown'].cummin()

all_stocks.head(5)


,Date,Code,Name,12m Low,12m High,Day Low,Day High,Day Price,Previous,Change,...,Adjusted Price,Sector,Next Day Price,Price Trend,Trend,Daily Return,Rolling Volatility,Cumulative Max,Drawdown,Max Drawdown So Far
0,2021-01-04,EGAD,Eaagads Ltd,8.2,14.0,12.5,12.5,12.5,12.5,0.0,...,0.0,Agricultural,365.00,1,Up,NaN,NaN,12.5,0.000000,0.000000
1,2021-01-04,KUKZ,Kakuzi Plc,300.0,397.0,365.0,365.0,365.0,365.0,0.0,...,0.0,Agricultural,78.00,0,Down,28.200000,NaN,365.0,0.000000,0.000000
2,2021-01-04,KAPC,Kapchorua Tea Kenya Plc,59.0,90.0,78.0,78.0,78.0,78.0,0.0,...,0.0,Agricultural,360.00,1,Up,-0.786301,NaN,365.0,-0.786301,-0.786301
3,2021-01-04,LIMT,Limuru Tea Plc,360.0,475.0,360.0,360.0,360.0,360.0,0.0,...,0.0,Agricultural,19.50,0,Down,3.615385,NaN,365.0,-0.013699,-0.786301
4,2021-01-04,SASN,Sasini Plc,14.8,20.0,19.5,19.5,19.5,19.5,0.0,...,0.0,Agricultural,129.75,1,Up,-0.945833,NaN,365.0,-0.946575,-0.946575
